In [ ]:
pip install unsloth datasets peft accelerate transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from unsloth import FastLanguageModel
from transformers import TrainingArguments, Trainer
from datasets import load_dataset
import torch


<ipython-input-3-778e4867ef2a>:1: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-2-7b-bnb-4bit",
    max_seq_length = 128,
    dtype = None,
    load_in_4bit = True,
)


==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/3.87G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/948 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8,
    lora_alpha = 16,
    lora_dropout = 0.05,
    target_modules = ["q_proj", "v_proj"],
    use_gradient_checkpointing = True,
    random_state = 42,
    use_rslora = False,
    loftq_config = None,
)


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.3.19 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files="/content/smiles-string-dataset2.csv")
dataset


DatasetDict({
    train: Dataset({
        features: ['smiles'],
        num_rows: 4178
    })
})

In [ ]:
def tokenize(example):
    return tokenizer(
        example["smiles"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )

tokenized = dataset.map(tokenize)
tokenized = tokenized["train"].train_test_split(test_size=0.1)


Map:   0%|          | 0/4178 [00:00<?, ? examples/s]

In [ ]:
# from transformers import Trainer, TrainingArguments

# # Define a custom collate_fn to add labels to the inputs
# def collate_fn(batch):
#     # Check the batch format
#     print(batch[0])  # Print the first example in the batch
#     inputs = tokenizer([example["smiles"] for example in batch], padding=True, truncation=True, return_tensors="pt")
#     inputs["labels"] = inputs["input_ids"].clone()  # Set labels equal to input_ids
#     return inputs


# FastLanguageModel.for_training(model)

# training_args = TrainingArguments(
#     output_dir="./llama-smiles",
#     per_device_train_batch_size=4,
#     gradient_accumulation_steps=2,
#     num_train_epochs=3,
#     logging_steps=10,
#     learning_rate=2e-4,
#     fp16=True,
#     save_total_limit=2,
#     optim="adamw_8bit",
#     report_to="none"
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized["train"],
#     eval_dataset=tokenized["test"],
#     data_collator=collate_fn,  # Pass the custom collate_fn to the Trainer
# )


In [ ]:
# trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,760 | Num Epochs = 3 | Total steps = 1,410
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 4,194,304/7,000,000,000 (0.06% trained)


ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,attention_mask.

In [ ]:
# from transformers import Trainer, TrainingArguments
# from datasets import Dataset

# # Assuming you've already loaded your dataset and tokenizer, for example:
# # dataset = your_loaded_dataset
# # tokenizer = your_tokenizer

# # Debugging step to inspect batch format
# def collate_fn(batch):
#     # Print the first example in the batch to check the format
#     print("Batch format:", batch[0])  # This will print the first example of the batch
#     inputs = tokenizer([example["smiles"] for example in batch], padding=True, truncation=True, return_tensors="pt")
#     inputs["labels"] = inputs["input_ids"].clone()  # Set labels equal to input_ids
#     return inputs

# # Convert the train dataset into the right format (if needed)
# train_dataset = Dataset.from_dict({"smiles": dataset["train"]["smiles"]})

# # Setting up the training arguments
# training_args = TrainingArguments(
#     output_dir="./llama-smiles",  # Directory to save checkpoints
#     per_device_train_batch_size=4,  # Batch size per device
#     gradient_accumulation_steps=2,  # Accumulate gradients over multiple steps
#     num_train_epochs=3,  # Number of epochs to train
#     logging_steps=10,  # Log every 10 steps
#     learning_rate=2e-4,  # Set the learning rate
#     fp16=True,  # Use mixed precision if available
#     save_total_limit=2,  # Limit the number of saved checkpoints
#     optim="adamw_8bit",  # Optimizer choice
#     report_to="none",  # Disable reporting
# )

# # Initialize the Trainer
# trainer = Trainer(
#     model=model,  # Pre-trained LLaMA model
#     tokenizer=tokenizer,  # Tokenizer
#     args=training_args,  # Training arguments
#     train_dataset=train_dataset,  # Training dataset
#     eval_dataset=tokenized["test"],  # Evaluation dataset (tokenized test set)
#     data_collator=collate_fn,  # Use the custom collate_fn for data processing
# )




<ipython-input-36-35f4ef904224>:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# # Start training
# trainer.train()

ValueError: No columns in the dataset match the model's forward method signature. The following columns have been ignored: [smiles]. Please check the dataset and model. You may need to set `remove_unused_columns=False` in `TrainingArguments`.

In [ ]:
from transformers import Trainer, TrainingArguments
from datasets import Dataset

# Debugging step to inspect batch format
def collate_fn(batch):
    # Print the first example in the batch to check the format
    print("Batch format:", batch[0])  # This will print the first example of the batch
    inputs = tokenizer([example["smiles"] for example in batch], padding=True, truncation=True, return_tensors="pt")
    inputs["labels"] = inputs["input_ids"].clone()  # Set labels equal to input_ids
    return inputs

# Convert the train dataset into the right format (if needed)
train_dataset = Dataset.from_dict({"smiles": dataset["train"]["smiles"]})

# Setting up the training arguments
training_args = TrainingArguments(
    output_dir="./llama-smiles",  # Directory to save checkpoints
    per_device_train_batch_size=4,  # Batch size per device
    gradient_accumulation_steps=2,  # Accumulate gradients over multiple steps
    num_train_epochs=3,  # Number of epochs to train
    logging_steps=10,  # Log every 10 steps
    learning_rate=2e-4,  # Set the learning rate
    fp16=True,  # Use mixed precision if available
    save_total_limit=2,  # Limit the number of saved checkpoints
    optim="adamw_8bit",  # Optimizer choice
    report_to="none",  # Disable reporting
    remove_unused_columns=False,  # Avoid removing unused columns (to keep 'smiles')
)

# Initialize the Trainer
trainer = Trainer(
    model=model,  # Pre-trained LLaMA model
    tokenizer=tokenizer,  # Tokenizer
    args=training_args,  # Training arguments
    train_dataset=train_dataset,  # Training dataset
    eval_dataset=tokenized["test"],  # Evaluation dataset (tokenized test set)
    data_collator=collate_fn,  # Use the custom collate_fn for data processing
)

# Start training
trainer.train()


<ipython-input-38-72714cf0da4d>:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,178 | Num Epochs = 3 | Total steps = 1,566
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 4,194,304/7,000,000,000 (0.06% trained)


Batch format: {'smiles': 'CCCCCCCN'}
Batch format: {'smiles': 'O=C1C=CC(=O)N1'}
Batch format: {'smiles': 'N#CC1CCCCC1n1cc(C(N)=O)c(Nc2cncc(Cl)c2)n1'}


Step,Training Loss
10,7.878700
20,5.332400
30,3.531100
40,2.921300
50,2.854000
60,2.609900
70,2.879500
80,2.554400
90,2.517800
100,2.526600


Batch format: {'smiles': 'CC1COC(=O)O1'}
Batch format: {'smiles': '[C-]#[N+]C1CCCCC1n1cc(C(N)=O)c(Nc2ccc(C(C)=NOC)cc2)n1'}
Batch format: {'smiles': 'Nc1ccc(S(=O)(=O)c2ccc(N)cc2)cc1'}
Batch format: {'smiles': 'c1ccc2c(c1)cnc1ccccc12'}
Batch format: {'smiles': 'CCC1(CC)C(=O)NC(=O)N(C)C1=O'}
Batch format: {'smiles': 'CC(=O)c1ccc(C)cc1'}
Batch format: {'smiles': 'CC(NC(=O)c1c[nH]c2ncc(C3CC3)nc12)C(=O)N1CCC1'}
Batch format: {'smiles': 'CC(O)(c1ccc(Nc2nn(C3CCC(O)CC3C#N)cc2C(N)=O)cc1)C(F)(F)F'}
Batch format: {'smiles': 'O=C(Nc1nc2cccc(-c3ccc(CN4CCCCC4)cc3)n2n1)C1CC1'}
Batch format: {'smiles': 'CC(C)(C)NC(=O)c1cncc(Nc2nn(C3CCCCC3C#N)cc2C(N)=O)c1'}
Batch format: {'smiles': 'NC(=O)NO'}
Batch format: {'smiles': 'CC(N)Cc1ccccc1'}
Batch format: {'smiles': 'CC1CO1'}
Batch format: {'smiles': 'O=C(NC1CCOCC1)c1cnn2ccc(N3CCCC3c3cc(F)ccc3F)nc12'}
Batch format: {'smiles': 'CC(Cl)(Cl)C(=O)O'}
Batch format: {'smiles': 'O=CCc1ccccc1'}
Batch format: {'smiles': 'O=C(O)COc1ccc(F)cc1'}
Batch format: {'smiles': '

TrainOutput(global_step=1566, training_loss=2.278671730523822, metrics={'train_runtime': 2824.7755, 'train_samples_per_second': 4.437, 'train_steps_per_second': 0.554, 'total_flos': 2.288684346384384e+16, 'train_loss': 2.278671730523822, 'epoch': 2.9952153110047846})

In [ ]:
model.save_pretrained("./smiles-lora")
tokenizer.save_pretrained("./smiles-lora")


('./smiles-lora/tokenizer_config.json',
 './smiles-lora/special_tokens_map.json',
 './smiles-lora/tokenizer.model',
 './smiles-lora/added_tokens.json',
 './smiles-lora/tokenizer.json')

In [ ]:
# FastLanguageModel.for_inference(model)

# prompt = "C1CC"  # A partial or starting SMILES string
# inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# # outputs = model.generate(
# #     **inputs,
# #     max_new_tokens=50,
# #     do_sample=True,
# #     top_p=0.95,
# #     temperature=0.7,
# #     repetition_penalty=1.1,
# # )
# outputs = model.generate(
#     **inputs,
#     max_new_tokens=50,
#     do_sample=True,
#     top_p=0.95,
#     temperature=0.7,
#     repetition_penalty=1.1,
#     eos_token_id=tokenizer.eos_token_id  # <- Important to prevent junk
# )


# print("Generated SMILES:", tokenizer.decode(outputs[0], skip_special_tokens=True))


Generated SMILES: C1CCCN1�����������������������������������������������


In [ ]:
# # Define your starting prompt
# import re

# prompt = "C1CC"
# inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# # Generate SMILES with controlled sampling
# outputs = model.generate(
#     **inputs,
#     max_new_tokens=50,
#     do_sample=True,
#     top_p=0.95,
#     temperature=0.7,
#     repetition_penalty=1.1,
#     eos_token_id=tokenizer.eos_token_id  # Stop early if EOS is generated
# )

# # Decode and clean the output
# raw_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

# # Optional: Filter to valid SMILES characters
# cleaned_smiles = re.sub(r'[^A-Za-z0-9@+\-\[\]\(\)=#$%.]', '', raw_output)

# print("Raw Generated SMILES:", raw_output)
# print("Cleaned SMILES:", cleaned_smiles)

Raw Generated SMILES: C1CC2C3C4CC5CC6=CC(=O)OC1C2CC3CC4CC5CC61���������������������
Cleaned SMILES: C1CC2C3C4CC5CC6=CC(=O)OC1C2CC3CC4CC5CC61


In [ ]:
outputs = model.generate(
    **inputs,
    max_new_tokens=50,
    do_sample=True,
    top_p=0.95,
    temperature=0.7,
    repetition_penalty=1.1,
    num_return_sequences=5  # Change this number to however many you want
)


In [ ]:
for i, output in enumerate(outputs):
    generated = tokenizer.decode(output, skip_special_tokens=True)
    print(f"SMILES {i + 1}: {generated}")


SMILES 1: C1CCC(=O)N(C2CCC(n3cc(C(N)=O)c(Nc4ccc(S(=O)(=O)C(F)(F)F)cc4)n3
SMILES 2: C1CC2(C=CC(C)C2)CC1C1CCC2(C#N)CC12�����������������������
SMILES 3: C1CCC(O)C(=O)NC1Nc1cc2nc[nH]c(Cl)c2c(Cl)c(Br)c1=O�����������
SMILES 4: C1CCCCC(=O)Nc1ccc(-c2ccnc(Nc3cn(C)n[nH]3)c2)cc1��������������
SMILES 5: C1CCCC1������������������������������������������������


In [ ]:
import re

# Assuming 'outputs' contains multiple generated sequences from the model
for i, output in enumerate(outputs):
    # Decode the output into a string
    generated = tokenizer.decode(output, skip_special_tokens=True)

    # Clean the decoded SMILES string by filtering invalid characters
    cleaned_smiles = re.sub(r'[^A-Za-z0-9@+\-\[\]\(\)=#$%.]', '', generated)

    # Print the cleaned SMILES
    print(f"SMILES {i + 1}: {cleaned_smiles}")


SMILES 1: C1CCC(=O)N(C2CCC(n3cc(C(N)=O)c(Nc4ccc(S(=O)(=O)C(F)(F)F)cc4)n3
SMILES 2: C1CC2(C=CC(C)C2)CC1C1CCC2(C#N)CC12
SMILES 3: C1CCC(O)C(=O)NC1Nc1cc2nc[nH]c(Cl)c2c(Cl)c(Br)c1=O
SMILES 4: C1CCCCC(=O)Nc1ccc(-c2ccnc(Nc3cn(C)n[nH]3)c2)cc1
SMILES 5: C1CCCC1


In [ ]:
# Define your starting prompt
import re

prompt = "CC(=O)OC1=CC=CC=C1C(O)=O"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# Generate SMILES with controlled sampling
outputs = model.generate(
    **inputs,
    max_new_tokens=50,
    do_sample=True,
    top_p=0.95,
    temperature=0.7,
    repetition_penalty=1.1,
    num_return_sequences=5  # Change this number to however many you want
)


In [ ]:
import re

# Assuming 'outputs' contains multiple generated sequences from the model
for i, output in enumerate(outputs):
    # Decode the output into a string
    generated = tokenizer.decode(output, skip_special_tokens=True)

    # Clean the decoded SMILES string by filtering invalid characters
    cleaned_smiles = re.sub(r'[^A-Za-z0-9@+\-\[\]\(\)=#$%.]', '', generated)

    # Print the cleaned SMILES
    print(f"SMILES {i + 1}: {cleaned_smiles}")


SMILES 1: CC(=O)OC1=CC=CC=C1C(O)=O
SMILES 2: CC(=O)OC1=CC=CC=C1C(O)=O
SMILES 3: CC(=O)OC1=CC=CC=C1C(O)=O
SMILES 4: CC(=O)OC1=CC=CC=C1C(O)=O
SMILES 5: CC(=O)OC1=CC=CC=C1C(O)=O
